In [1]:
!git clone https://github.com/NicolasCambon/RNN_UniversalDependencies_FR.git #Téléchargment des données

Cloning into 'RNN_UniversalDependencies_FR'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 43 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [23]:
import numpy as np

def donne(fic):
  print("lecture des donnees d'entrée depuis le fichier : ", fic) 
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]

  POP=[]
  Out=[]
  for i in range(len(sentence)):  
    pa=[]
    no=[]
    if i <= len(sentence)//2:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'

        if "Sing" in sentence[i][e][5]:
          nomb+="_S"
        elif "Plur" in sentence[i][e][5]:
          nomb+="_P"

        pa.append(sentence[i][e][3]+inf+nomb)
        if sentence[i][e][3] != "VERB":
          no.append("2")
        if sentence[i][e][3] == "VERB" and len(nomb) != 0:
          no.append("1")
        if sentence[i][e][3] == "VERB" and len(nomb) == 0:
          no.append("2")
      POP.append(pa)
      Out.append(no)
      
    else:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'
            
        if sentence[i][e][3] != "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_S"
          elif "Plur" in sentence[i][e][5]:
            nomb+="_P"
          pa.append(sentence[i][e][3]+inf+nomb)
          no.append("2")
        elif sentence[i][e][3] == "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_P"
          elif "Plur" in sentence[i][e][5]:
            nomb+="_S"
          pa.append(sentence[i][e][3]+inf+nomb)
          if len(nomb) != 0:
            no.append("0")
          else: #elif len(nomb) == 0:
            no.append("2")
           
      POP.append(pa)
      Out.append(no)
  return(POP,Out) 


def maximun(POP_tr,POP_te):
  maxi = len(POP_tr[0])
  for i in POP_tr:
    if len(i) > maxi:
      maxi=len(i)
  for j in POP_te:
    if len(j) > maxi:
      maxi=len(j)
  return(maxi)

def rempli(POP_tr,POP_te,Out_tr,Out_te,maxi):
  
  for i in range (len(POP_tr)):
    if len(POP_tr[i]) < maxi:
      for k in range(maxi-len(POP_tr[i])):
        POP_tr[i].append('Remplisseur')
        Out_tr[i].append(2)
  
  for j in range (len(POP_te)):
    if len(POP_te[j]) < maxi:
      for k in range(maxi-len(POP_te[j])):
        POP_te[j].append('Remplisseur')
        Out_te[j].append(2)
        
  return(POP_tr,Out_tr,POP_te,Out_te)

def dico(POP_tr,POP_te):
  dic={}
  lis=[]
  k=1
  for i in range (len(POP_tr)):
    for j in range(len(POP_tr[i])):
      if POP_tr[i][j] not in lis:
        lis.append(POP_tr[i][j])
        dic[POP_tr[i][j]]=k
        k+=1
        
  for i in range (len(POP_te)):
    for j in range(len(POP_te[i])):
      if POP_te[i][j] not in lis:
        lis.append(POP_te[i][j])
        dic[POP_te[i][j]]=k
        k+=1  
  return(dic)

def mise_en_forme(POP,Out,dic):
  Input=[]
  for i in range (len(POP)):
    intp=[]
    for j in range(len(POP[i])):
      intp.append(dic[POP[i][j]])
    Input.append(intp)
  
  POP_r = np.array(Input).reshape (len(Input),maxi,1)
  Out_r = np.array(Out).reshape (len(Out),maxi)
  return(POP_r,Out_r)


inp_tr,Out_tr= donne('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt')
inp_te,Out_te= donne('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt')
maxi = maximun(inp_tr,inp_te)
inp_tr_r,Out_tr_r,inp_te_r,Out_te_r= rempli(inp_tr,inp_te,Out_tr,Out_te,maxi)
dictionnaire=dico(inp_tr_r,inp_te_r)



x_train,y_train = mise_en_forme(inp_tr_r,Out_tr_r,dictionnaire)
print('x shape = ', x_train.shape)
print('y shape = ', y_train.shape)
x_test,y_test = mise_en_forme(inp_te_r,Out_te_r,dictionnaire)
print('x shape = ', x_test.shape)
print('y shape = ', y_test.shape)

lecture des donnees d'entrée depuis le fichier :  RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt
lecture des donnees d'entrée depuis le fichier :  RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt
x shape =  (2230, 150, 1)
y shape =  (2230, 150)
x shape =  (455, 150, 1)
y shape =  (455, 150)


In [0]:
def distance_verbe(fic):
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]
  num=0
  deno=0
  for i in range (len(sentence)):
    lis=[]
    for j in range(len(sentence[i])):
      if len(sentence[i][j])>8 and sentence[i][j][7]== 'root':
        chiffre = sentence[i][j][0]
        for k in range(len(sentence[i])):
          if len(sentence[i][k])>8 and sentence[i][k][6]== chiffre and 'nsubj'in sentence[i][k][7] and sentence[i][k][0] not in lis:
            lis.append(sentence[i][k][0])
            num += int(chiffre)-int(sentence[i][k][0])
            deno+=1
  return (num/deno)

distance_tr = distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt')
distance_te = distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt')
         

In [37]:
import keras
from keras.models import Sequential
from keras.layers import LSTM




model = Sequential()  
model.add(LSTM(maxi,input_shape=(x_train.shape[1:])))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test, y_test))





Train on 2230 samples, validate on 455 samples
Epoch 1/10
2230/2230 [==============================] - 12s 5ms/step - loss: 3.2736 - acc: 0.0000e+00 - val_loss: 2.3033 - val_acc: 0.0000e+00
Epoch 2/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.6023 - acc: 0.0000e+00 - val_loss: 1.1118 - val_acc: 0.9868
Epoch 3/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.0538 - acc: 0.9830 - val_loss: 1.0347 - val_acc: 0.9231
Epoch 4/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.0353 - acc: 0.9063 - val_loss: 1.0347 - val_acc: 0.9868
Epoch 5/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.0353 - acc: 0.9910 - val_loss: 1.0347 - val_acc: 0.9868
Epoch 6/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.0353 - acc: 0.9915 - val_loss: 1.0347 - val_acc: 0.9868
Epoch 7/10
2230/2230 [==============================] - 10s 5ms/step - loss: 1.0353 - acc: 0.9915 - val_loss: 1.0347 - val_acc: 0.9